In [1]:
import os
import json
import requests
import pandas as pd
from pathlib import Path

In [11]:
pd.options.display.max_rows=400
pd.options.display.max_colwidth=100

In [5]:
ref_file_path = Path('Data/opencharge_reference_data.json')
with ref_file_path.open('r', encoding='utf8') as f:
    ref_data = json.load(f)

In [8]:
conn_types_df = pd.DataFrame(ref_data['ConnectionTypes'])
conn_types_df.sort_values('ID', inplace=True)
conn_types_df = conn_types_df[['ID','Title', 'FormalName', 'IsDiscontinued', 'IsObsolete']]
conn_types_df.head()

,ID,Title,FormalName,IsDiscontinued,IsObsolete
39,0,Unknown,Not Sp...,None,None
34,1,Type 1...,SAE J1...,None,None
10,2,CHAdeMO,IEC 62...,None,None
2,3,BS1363...,BS1363...,None,None
1,4,Blue C...,None,None,None


In [12]:
curr_types_df = pd.DataFrame(ref_data['CurrentTypes'])
curr_types_df.sort_values('ID', inplace=True)
curr_types_df = curr_types_df[['ID','Title', 'Description']]
curr_types_df.head()


,ID,Title,Description
0,10,AC (Single-Phase),Alternating Current - Single Phase
1,20,AC (Three-Phase),Alternating Current - Three Phase
2,30,DC,Direct Current


In [16]:
operators_df = pd.DataFrame(ref_data['Operators'])
operators_df.sort_values('ID', inplace=True)
operators_df.drop(['WebsiteURL', 'PhonePrimaryContact', 'PhoneSecondaryContact', 'AddressInfo',
                  'BookingURL', 'ContactEmail', 'FaultReportEmail', 'IsRestrictedEdit'], axis=1, inplace=True)
operators_df = operators_df[['ID', 'Title', 'IsPrivateIndividual', 'Comments']]
operators_df.head()


,ID,Title,IsPrivateIndividual,Comments
2,1,(Unknown Operator),None,None
202,2,Elektrobay (UK),False,None
536,3,POD Point (UK),False,Part of Groupe EDF
96,5,ChargePoint,False,"GB*CPI,AT*HTB"
480,6,Nissan US Dealer Network,None,None


In [17]:
usage_df = pd.DataFrame(ref_data['UsageTypes'])
usage_df.sort_values('ID', inplace=True)
usage_df = usage_df[['ID', 'Title', 'IsPayAtLocation','IsMembershipRequired','IsAccessKeyRequired']]
usage_df.head()


,ID,Title,IsPayAtLocation,IsMembershipRequired,IsAccessKeyRequired
0,0,(Unknown),None,None,None
4,1,Public,None,None,None
2,2,Private - Restricted Access,None,True,None
3,3,Privately Owned - Notice Required,None,None,None
5,4,Public - Membership Required,False,True,True


In [22]:
status_types_df = pd.DataFrame(ref_data['StatusTypes'])
status_types_df.sort_values('ID', inplace=True)
status_types_df = status_types_df[['ID', 'Title', 'IsOperational','IsUserSelectable']]

print(status_types_df.shape)
status_types_df.head()

(10, 4)


,ID,Title,IsOperational,IsUserSelectable
0,0,Unknown,None,True
1,10,Currently Available (Automated Status),True,False
2,20,Currently In Use (Automated Status),True,False
3,30,Temporarily Unavailable,True,True
4,50,Operational,True,True


In [23]:
# Exportdataframes to CSV for database imports
conn_types_df.to_csv('Data/reference_data_connection_types.csv', encoding='utf8', header=True, index=False)
curr_types_df.to_csv('Data/reference_data_current_types.csv', encoding='utf8', header=True, index=False)
operators_df.to_csv('Data/reference_data_operators_types.csv', encoding='utf8', header=True, index=False)
usage_df.to_csv('Data/reference_data_usage_types.csv', encoding='utf8', header=True, index=False)
status_types_df.to_csv('Data/reference_status_connection_types.csv', encoding='utf8', header=True, index=False)


In [25]:
# Rebuild reference data json file
new_ref_json = {
                'ConnectionTypes':conn_types_df.to_dict(orient='records'),
                'CurrentTypes':curr_types_df.to_dict(orient='records'),
                'OperatorTypes':operators_df.to_dict(orient='records'),
                'UsageTypes':usage_df.to_dict(orient='records'),
                'StatusTypes':status_types_df.to_dict(orient='records')
               }
with open('Data/new_reference_data.json', 'w') as f:
    json.dump(new_ref_json, f, indent=4)
    